In [ ]:
import mysql_connection
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.impute import SimpleImputer

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import GradientBoostingRegressor
import pandas as pd
import joblib


def fetch_data(team, players):
    connection = mysql_connection.get_mysql_connection()

    if connection:
        dfs = []

        for player in players:
            if team == 'All' and player == 'All':
                sql_query = """
                SELECT DISTINCT b.match_id, b.team, b.opposite_team, b.player, b.batting_position, b.dismissal_method, b.runs, b.balls, b.fours, b.sixes, b.strike_rate, m.ground, m.match_date
                FROM cricket_info.batting AS b
                JOIN cricket_info.matches AS m ON b.match_id = m.match_id
                WHERE b.batting_position != '-' AND (b.balls != '-' AND b.balls != '0');
                """
                cursor = connection.cursor()
                cursor.execute(sql_query)
            elif team == 'All':
                sql_query = """
                SELECT DISTINCT b.match_id, b.team, b.opposite_team, b.player, b.batting_position, b.dismissal_method, b.runs, b.balls, b.fours, b.sixes, b.strike_rate, m.ground, m.match_date
                FROM cricket_info.batting AS b
                JOIN cricket_info.matches AS m ON b.match_id = m.match_id
                WHERE  b.batting_position != '-' AND b.player LIKE %s AND (b.balls != '-' AND b.balls != '0');
                """
                cursor = connection.cursor()
                cursor.execute(sql_query, (f'%{player}%',))
            else:
                sql_query = """
                SELECT DISTINCT b.match_id, b.team, b.opposite_team, b.player, b.batting_position, b.dismissal_method, b.runs, b.balls, b.fours, b.sixes, b.strike_rate, m.ground, m.match_date
                FROM cricket_info.batting AS b
                JOIN cricket_info.matches AS m ON b.match_id = m.match_id
                WHERE b.batting_position != '-' AND  b.team = %s AND b.player LIKE %s AND (b.balls != '-' AND b.balls != '0') ;
                """
                cursor = connection.cursor()
                cursor.execute(sql_query, (team, f'%{player}%'))

           
            rows = cursor.fetchall()
            df = pd.DataFrame(rows, columns=['match_id', 'team', 'opposite_team', 'player', 'batting_position', 'dismissal_method', 'runs', 'balls', 'fours', 'sixes', 'strike_rate', 'ground', 'date'])
            dfs.append(df)
            cursor.close()

        connection.close()
        df = pd.concat(dfs, ignore_index=True)
        return df
    else:
        print("Connection to the database failed.")
        return None


# Function to convert string representation of overs to decimal
def convert_to_decimal(overs_str):
    if '.' in overs_str:
        overs, balls = overs_str.split('.')
        decimal_overs = float(overs) + float(balls) / 6
    else:
        decimal_overs = float(overs_str)
    return decimal_overs


def transform_data(df):
    # Preprocessing part starts here...
    df['date'] = pd.to_datetime(df['date'], format='%b %d, %Y', errors='coerce')
    df['runs'] = pd.to_numeric(df['runs'], errors='coerce')
    df['balls'] = pd.to_numeric(df['balls'], errors='coerce')
    df['fours'] = pd.to_numeric(df['fours'], errors='coerce')
    df['sixes'] = pd.to_numeric(df['sixes'], errors='coerce')
    df['strike_rate'] = pd.to_numeric(df['strike_rate'], errors='coerce')
    
    try:
        df['day'] = df['date'].dt.day
        df['month'] = df['date'].dt.month
        df['year'] = df['date'].dt.year
    except ValueError:
        print("Dropping records with incorrect date format...")
        wrong_date_indices = df.index[(df['date'].dt.day.isna()) | (df['date'].dt.month.isna()) | (df['date'].dt.year.isna())]
        df.drop(wrong_date_indices, inplace=True)
        print(f"Dropped {len(wrong_date_indices)} records.")

    return df


from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd

def old_models_evaluate(df):
    master_df = df.copy()  # Always work on a copy to avoid unintended modifications to the original DataFrame

    # Drop records with non-numeric values in 'day', 'month', or 'year' columns
    master_df = master_df.dropna(subset=['day', 'month', 'year'], how='any')
    
    # Convert 'day', 'month', and 'year' columns to numeric
    master_df['day'] = pd.to_numeric(master_df['day'], errors='coerce')
    master_df['month'] = pd.to_numeric(master_df['month'], errors='coerce')
    master_df['year'] = pd.to_numeric(master_df['year'], errors='coerce')
    
    # Drop records with NaN values after conversion
    master_df = master_df.dropna(subset=['day', 'month', 'year'], how='any')

    # Ensure 'team', 'opposite_team', and 'ground' columns are of string data type
    master_df[['team', 'opposite_team', 'ground']] = master_df[['team', 'opposite_team', 'ground']].astype(str)
    
    # Remove records with non-numeric or NaN values in input features or target variable
    numeric_features = ['runs', 'balls', 'fours', 'sixes', 'strike_rate']
    master_df = master_df.dropna(subset=numeric_features + ['dismissal_method'], how='any')

    # Splitting the data into features (X) and target variable (y)
    X = master_df[['team', 'opposite_team', 'runs', 'balls', 'fours', 'sixes', 'strike_rate', 'ground', 'day', 'month', 'year']]
    y = master_df['dismissal_method']
    
    # Encode target variable using LabelEncoder
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    
    # Splitting the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)
    
    # Preprocessing: Define numeric and categorical transformers
    numeric_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='mean')), ('scaler', StandardScaler())])
    categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
    
    preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, ['runs', 'balls', 'fours', 'sixes', 'strike_rate', 'day', 'month', 'year']),
                                                   ('cat', categorical_transformer, ['team', 'opposite_team', 'ground'])])
    
    # Initialize regression models
    models = [
        ('Linear Regression', LinearRegression()),
        ('Support Vector Regression', SVR()),
        ('Random Forest Regression', RandomForestRegressor(random_state=42)),
        ('Gradient Boosting Regression', GradientBoostingRegressor(random_state=42)),
        ('Decision Tree Regression', DecisionTreeRegressor(random_state=42)),
    ]

    # Train and evaluate models
    for name, model in models:
        classifier = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', model)])
        classifier.fit(X_train, y_train)
        predictions = classifier.predict(X_test)
        mse = mean_squared_error(y_test, predictions)
        rmse = mse ** 0.5
        r2 = r2_score(y_test, predictions)
        print(f'{name}:')
        print(f'Mean Squared Error: {mse}')
        print(f'Root Mean Squared Error: {rmse}')
        print(f'R-squared: {r2}')
        print('\n')

# Example usage with your DataFrame
# models_evaluate(df)


def models_evaluate(df):
    master_df = df.copy()  # Always work on a copy to avoid unintended modifications to the original DataFrame

    # Drop records with non-numeric values in 'day', 'month', or 'year' columns
    master_df = master_df.dropna(subset=['day', 'month', 'year'], how='any')
    
    # Convert 'day', 'month', and 'year' columns to numeric
    master_df['day'] = pd.to_numeric(master_df['day'], errors='coerce')
    master_df['month'] = pd.to_numeric(master_df['month'], errors='coerce')
    master_df['year'] = pd.to_numeric(master_df['year'], errors='coerce')
    
    # Drop records with NaN values after conversion
    master_df = master_df.dropna(subset=['day', 'month', 'year'], how='any')

    # Ensure 'team', 'opposite_team', and 'ground' columns are of string data type
    master_df[['team', 'opposite_team', 'ground']] = master_df[['team', 'opposite_team', 'ground']].astype(str)
    
    # Remove records with non-numeric or NaN values in input features or target variable
    numeric_features = ['balls', 'fours', 'sixes', 'strike_rate', 'day', 'month', 'year']
    master_df = master_df.dropna(subset=numeric_features + ['dismissal_method', 'runs'], how='any')

    # Splitting the data into features (X) and target variable (y)
    X = master_df[['team', 'opposite_team', 'dismissal_method', 'balls', 'fours', 'sixes', 'strike_rate', 'ground', 'day', 'month', 'year']]
    y = master_df['runs']
    
    # Splitting the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Preprocessing: Define numeric and categorical transformers
    numeric_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='mean')), ('scaler', StandardScaler())])
    categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
    
    preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, numeric_features),
                                                   ('cat', categorical_transformer, ['team', 'opposite_team', 'dismissal_method', 'ground'])])
    
    # Initialize regression models
    models = [
        ('Linear Regression', LinearRegression()),
        ('Support Vector Regression', SVR()),
        ('Random Forest Regression', RandomForestRegressor(random_state=42)),
        ('Gradient Boosting Regression', GradientBoostingRegressor(random_state=42)),
        ('Decision Tree Regression', DecisionTreeRegressor(random_state=42)),
    ]

    # Train and evaluate models
    for name, model in models:
        regressor = Pipeline(steps=[('preprocessor', preprocessor), ('regressor', model)])
        regressor.fit(X_train, y_train)
        predictions = regressor.predict(X_test)
        mse = mean_squared_error(y_test, predictions)
        rmse = mse ** 0.5
        r2 = r2_score(y_test, predictions)
        print(f'{name}:')
        print(f'Mean Squared Error: {mse}')
        print(f'Root Mean Squared Error: {rmse}')
        print(f'R-squared: {r2}')
        print('\n')


def model_implement(df):
    master_df = df.copy()  # Always work on a copy to avoid unintended modifications to the original DataFrame

    # Drop records with non-numeric values in 'day', 'month', or 'year' columns
    master_df = master_df.dropna(subset=['day', 'month', 'year'], how='any')
    
    # Convert 'day', 'month', and 'year' columns to numeric
    master_df['day'] = pd.to_numeric(master_df['day'], errors='coerce')
    master_df['month'] = pd.to_numeric(master_df['month'], errors='coerce')
    master_df['year'] = pd.to_numeric(master_df['year'], errors='coerce')
    
    # Drop records with NaN values after conversion
    master_df = master_df.dropna(subset=['day', 'month', 'year'], how='any')

    # Ensure 'team', 'opposite_team', and 'ground' columns are of string data type
    master_df[['team', 'opposite_team', 'ground']] = master_df[['team', 'opposite_team', 'ground']].astype(str)
    
    # Remove records with non-numeric or NaN values in input features or target variable
    numeric_features = ['balls', 'fours', 'sixes', 'strike_rate', 'day', 'month', 'year']
    master_df = master_df.dropna(subset=numeric_features + ['dismissal_method', 'runs'], how='any')

    # Splitting the data into features (X) and target variable (y)
    X = master_df[['team', 'opposite_team', 'dismissal_method', 'balls', 'fours', 'sixes', 'strike_rate', 'ground', 'day', 'month', 'year']]
    y = master_df['runs']
    
    # Splitting the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Preprocessing: Define numeric and categorical transformers
    numeric_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='mean')), ('scaler', StandardScaler())])
    categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
    
    preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, numeric_features),
                                                   ('cat', categorical_transformer, ['team', 'opposite_team', 'dismissal_method', 'ground'])])
    
    # Initialize Random Forest Regression model
    model = RandomForestRegressor(random_state=42)
    
    # Create pipeline
    pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('regressor', model)])
    
    # Train the model
    pipeline.fit(X_train, y_train)
    
    # Save the trained model for future use
    joblib.dump(pipeline, 'random_forest_regressor_model.pkl')
    
    # Evaluate the model
    predictions = pipeline.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    rmse = mse ** 0.5
    r2 = r2_score(y_test, predictions)
    print('Random Forest Regression:')
    print(f'Mean Squared Error: {mse}')
    print(f'Root Mean Squared Error: {rmse}')
    print(f'R-squared: {r2}')





if __name__ == "__main__":
    team = 'All'
    players = ['All']
    # team = 'New Zealand'
    # players = ['Tim Southee', 'Ish Sodhi']
    
    df = fetch_data(team, players)
    print(df)
    transformed_df = transform_data(df)
    print(transformed_df)
    print(transformed_df.columns)
    # models_evaluate(transformed_df)
    model_implement(transformed_df)

Connected to MySQL database
          match_id          team opposite_team              player  \
0         T20I # 3  South Africa   New Zealand        Graeme Smith   
1         T20I # 3  South Africa   New Zealand      Jacques Kallis   
2         T20I # 3  South Africa   New Zealand    Herschelle Gibbs   
3         T20I # 3  South Africa   New Zealand         Justin Kemp   
4         T20I # 3  South Africa   New Zealand        Mark Boucher   
...            ...           ...           ...                 ...   
40387  T20I # 2427        U.A.E.   Afghanistan  Azmatullah Omarzai   
40388  T20I # 2427        U.A.E.   Afghanistan     Darwish Rasooli   
40389  T20I # 2427        U.A.E.   Afghanistan   Najibullah Zadran   
40390  T20I # 2427        U.A.E.   Afghanistan       Mohammad Nabi   
40391  T20I # 2427        U.A.E.   Afghanistan  Sharafuddin Ashraf   

      batting_position   dismissal_method runs balls fours sixes strike_rate  \
0                    1             caught   61    4